In [1]:
!pip install sentence_transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 2.0 MB/s eta 0:00:0000:0100:01m


In [2]:
import pandas as pd
import numpy as np
import json
from sentence_transformers import SentenceTransformer


/opt/conda/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
2024-08-01 21:40:08.576900: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-01 21:40:08.577019: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-01 21:40:08.707055: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
df = pd.read_parquet("hf://datasets/hakeematyab/HealthCareMagicWithSummary-100k/data/train-00000-of-00001.parquet")
df.head()

,input,preprocessed_output,summarized_input,summarized_output
0,I woke up this morning feeling the whole room ...,"Hi, Thank you for posting your query. The most...",This morning I woke up feeling dizzy and nause...,"According to the doctor, the most likely cause..."
1,My baby has been pooing 5-6 times a day for a ...,Hi... Thank you for consulting in CareConnect....,My baby has been pooing 5-6 times a day for a ...,Your kid has viral diarrhea. It will take 5-7 ...
2,"Hello, My husband is taking Oxycodone due to a...","Hello, and I hope I can help you today.First, ...",My husband is taking Oxycodone for a broken le...,"According to the doctor, there is no medicatio..."
3,lump under left nipple and stomach pain (male)...,HI. You have two different problems. The lump ...,A male asks for help with a lump under his nip...,The lump under the nipple should be removed an...
4,I have a 5 month old baby who is very congeste...,Thank you for using CareConnect. I would sugge...,My baby has a terrible cough. She started chok...,Your baby might have bronchiolitis. Albuterol ...


In [4]:
chunks_Df = pd.DataFrame()

In [5]:
print(df['summarized_input'].loc[0]+" "+df['summarized_output'].loc[0])

This morning I woke up feeling dizzy and nauseous. After taking Panadol and sleeping, the dizziness subsides. According to the doctor, the most likely cause of your symptoms is benign paroxysmal positional vertigo. Betahistine tablets and exercise will help relieve the symptoms. An ENT evaluation is recommended.


In [6]:
chunks_Df['chunk'] = df['summarized_input']+" "+df['summarized_output']

In [7]:
print(chunks_Df.shape)
print(chunks_Df.head())


(112165, 1)
                                               chunk
0  This morning I woke up feeling dizzy and nause...
1  My baby has been pooing 5-6 times a day for a ...
2  My husband is taking Oxycodone for a broken le...
3  A male asks for help with a lump under his nip...
4  My baby has a terrible cough. She started chok...


In [8]:
print(chunks_Df.isnull().sum())

chunk    0
dtype: int64


In [9]:
df2 = pd.read_json("hf://datasets/RafaelMPereira/HealthCareMagic-100k-Chat-Format-en/HealthCareMagic-100k-en.jsonl", lines=True)


In [10]:
df2.head()
df2.rename(columns={'text':'chunk'},inplace=True)
df2.head()

,chunk
0,<human>: I woke up this morning feeling the wh...
1,<human>: My baby has been pooing 5-6 times a d...
2,"<human>: Hello, My husband is taking Oxycodone..."
3,<human>: lump under left nipple and stomach pa...
4,<human>: I have a 5 month old baby who is very...


In [11]:
chunks_Df = pd.concat([chunks_Df,df2], ignore_index=True)

In [12]:
len(chunks_Df)

224330

In [13]:
print(chunks_Df.isnull().sum())

chunk    0
dtype: int64


In [14]:
df3 = pd.read_json("hf://datasets/FreedomIntelligence/Disease_Database/disease_database_en.json")

In [15]:
print(df3.head())

            disease_id                    disease  \
0  1656164150939770880     Carotid Sinus Syndrome   
1  1656164157617102848      Cranial Osteomyelitis   
2  1656164143901728768  Tuberculous Osteomyelitis   
3  1656164192798924800              Lipodystrophy   
4  1656164179163242496           Vulvar Pemphigus   

                                      common_symptom  \
0  Syncope, Cardiac Arrest, Dizziness, Fatigue, T...   
1  Headache and fever, localized scalp redness, s...   
2  Low-grade fever, fatigue, night sweats, loss o...   
3  Hematuria, pyuria, acute abdominal pain, edema...   
4  Blisters, pain, anorexia, mucosal damage, hype...   

                                           treatment  
0  The treatment strategies for Carotid Sinus Syn...  
1  What are the treatment methods for cranial ost...  
2  The modalities for treating Tuberculous Osteom...  
3  There are currently no effective treatments fo...  
4  Treatment for Vulvar Pemphigus includes severa...  


In [16]:
df3['chunk'] = "Disease: "+df3['disease']+" Common Symptoms: "+df3["common_symptom"]+" Treatment: "+df3['treatment']

In [17]:
print(df3['chunk'].head())

0    Disease: Carotid Sinus Syndrome Common Symptom...
1    Disease: Cranial Osteomyelitis Common Symptoms...
2    Disease: Tuberculous Osteomyelitis Common Symp...
3    Disease: Lipodystrophy Common Symptoms: Hematu...
4    Disease: Vulvar Pemphigus Common Symptoms: Bli...
Name: chunk, dtype: object


In [18]:
print(df3.isnull().sum())

disease_id        0
disease           0
common_symptom    0
treatment         0
chunk             0
dtype: int64


In [19]:
chunks_Df = pd.concat([chunks_Df,pd.DataFrame(df3['chunk'])], ignore_index=True)

In [20]:
print(chunks_Df.isnull().sum())

chunk    0
dtype: int64


In [21]:
df4 = pd.read_parquet("hf://datasets/lavita/ChatDoctor-HealthCareMagic-100k/data/train-00000-of-00001-5e7cb295b9cff0bf.parquet")


In [22]:
df4['chunk'] = "Patient: "+df4['input']+" Doctor: "+df4['output']
df4['chunk'].head()

0    Patient: I woke up this morning feeling the wh...
1    Patient: My baby has been pooing 5-6 times a d...
2    Patient: Hello, My husband is taking Oxycodone...
3    Patient: lump under left nipple and stomach pa...
4    Patient: I have a 5 month old baby who is very...
Name: chunk, dtype: object

In [23]:
chunks_Df = pd.concat([chunks_Df,pd.DataFrame(df4['chunk'])], ignore_index=True)

In [24]:
print(chunks_Df.isnull().sum())

chunk    0
dtype: int64


In [25]:
df5 = pd.read_parquet("hf://datasets/ruslanmv/ai-medical-chatbot/dialogues.parquet")
print(df5.head())


                                         Description  \
0      Q. What does abutment of the nerve root mean?   
1  Q. What should I do to reduce my weight gained...   
2  Q. I have started to get lots of acne on my fa...   
3  Q. Why do I have uncomfortable feeling between...   
4  Q. My symptoms after intercourse threatns me e...   

                                             Patient  \
0  Hi doctor,I am just wondering what is abutting...   
1  Hi doctor, I am a 22-year-old female who was d...   
2  Hi doctor! I used to have clear skin but since...   
3  Hello doctor,I am having an uncomfortable feel...   
4  Hello doctor,Before two years had sex with a c...   

                                              Doctor  
0  Hi. I have gone through your query with dilige...  
1  Hi. You have really done well with the hypothy...  
2  Hi there Acne has multifactorial etiology. Onl...  
3  Hello. The popping and discomfort what you fel...  
4  Hello. The HIV test uses a finger prick blood ..

In [26]:
df5['chunk'] = "Description: "+df5['Description']+" Patient: "+df5['Patient']+" Doctor: "+df5["Doctor"]
df5['chunk'].head()

0    Description: Q. What does abutment of the nerv...
1    Description: Q. What should I do to reduce my ...
2    Description: Q. I have started to get lots of ...
3    Description: Q. Why do I have uncomfortable fe...
4    Description: Q. My symptoms after intercourse ...
Name: chunk, dtype: object

In [27]:
chunks_Df = pd.concat([chunks_Df,pd.DataFrame(df5['chunk'])], ignore_index=True)
print(chunks_Df.isnull().sum())

chunk    0
dtype: int64


In [28]:
chunks_Df.shape

(603031, 1)

In [29]:
!pip install datasets
from datasets import load_dataset

ds = load_dataset("omi-health/medical-dialogue-to-soap-summary")

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


Generating train split:   0%|          | 0/9250 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/500 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/250 [00:00<?, ? examples/s]

In [30]:
df6 = pd.DataFrame(ds["train"])


In [31]:
df6 = pd.concat([df6,pd.DataFrame(ds["test"])],ignore_index=True)

In [32]:
df6.rename(columns={'dialogue':'chunk'},inplace=True)

In [33]:
chunks_Df = pd.concat([chunks_Df,pd.DataFrame(df6['chunk'])], ignore_index=True)
print(chunks_Df.isnull().sum())

chunk    0
dtype: int64


In [34]:
print(chunks_Df.head())
print(chunks_Df.shape)

                                               chunk
0  This morning I woke up feeling dizzy and nause...
1  My baby has been pooing 5-6 times a day for a ...
2  My husband is taking Oxycodone for a broken le...
3  A male asks for help with a lump under his nip...
4  My baby has a terrible cough. She started chok...
(612531, 1)


In [35]:
chunks_Df.duplicated().sum()

10378

In [36]:
chunks_Df.drop_duplicates(inplace=True)

In [37]:
chunks_Df.duplicated().sum()
chunks_Df.isnull().sum()

chunk    0
dtype: int64

In [38]:
chunks_Df.shape

(602153, 1)

In [39]:
model = SentenceTransformer("neuml/pubmedbert-base-embeddings")

def create_embeddings(dataset):
    # Generate embeddings for the 'text'
    chunks_list = dataset.tolist()
    embeddings = model.encode(chunks_list)

    return embeddings


embeddings = create_embeddings(chunks_Df['chunk'])

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/6.12k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/667 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/706k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/74.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/18818 [00:00<?, ?it/s]

In [43]:
!pip install pinecone-client

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [50]:
print(len(converted_embeddings))
print(len(converted_embeddings[0]))

602153
768


In [61]:
import numpy as np
from pinecone import Pinecone

# Assuming your vectors are in a numpy array called 'vectors' with shape (602153, 768)

# Initialize Pinecone
pc = Pinecone(api_key="c4c9953d-f1cf-4ccc-aef6-3689d0991f94")
index = pc.Index("ai-doctor")

# Process vectors in batches
batch_size = 100  # You can adjust this based on your needs
for i in range(0, embeddings.shape[0], batch_size):
    batch = embeddings[i:i+batch_size]
    
    # Create a list of dictionaries for the batch
    vector_list = [
        {
            'id': f'vec_{j}',  # Create a unique ID for each vector
            'values': vec.tolist(),  # Convert numpy array to list
        }
        for j, vec in enumerate(batch, start=i)
    ]
    
    # Upsert the batch
    index.upsert(vectors=vector_list)

print("Upsert complete")

Upsert complete


In [63]:
from zipfile import ZipFile


In [65]:
with ZipFile('embeddings.zip','w') as zip: 
        # writing each file one by one 
        zip.write('/kaggle/working/embeddings') 

In [64]:
!pip install pickle
import pickle
# open a file, where you ant to store the data
file = open('embeddings', 'wb')

# dump information to that file
pickle.dump(embeddings, file)


file.close()

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


ERROR: Could not find a version that satisfies the requirement pickle (from versions: none)
ERROR: No matching distribution found for pickle


TypeError: stat: path should be string, bytes, os.PathLike or integer, not BufferedWriter